We will try to work on different tasks and their models to fine tune them

In [1]:
!pip install transformers==4.28.0 accelerate > /dev/null

#The Details related to BERT

How the input is expected by Bert? 

- Sequence of Words. So need to be tokenised

What special tokens are used 

- SEP to tell where the sentences end, with respect to kind of data

- CLS is first of every token that means classification token

Max Number of token fed into BERT is 512. Rest needs to be padded

BERT tokenizer output vectors  are of 768 size. They can be used as inputs for various tasks

In [2]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

In [3]:
import tensorflow as tf
import pandas as pd
import json
import gc

from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('english')

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!wget https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv

In [5]:
data_path = "/content/bbc-text.csv"

In [6]:
data = pd.read_csv(data_path)
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [7]:
data['count'] = data['text'].apply(lambda x: len(x.split(' ')))

In [9]:
data['encoded'] = data['category'].astype("category").cat.codes

In [10]:
data_texts = data['text'].to_list()

data_labels = data['encoded'].to_list()

In [11]:
data_texts[0]

'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to high

In [12]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [14]:
#the data is shuffled when they are seperated into train,test and validation

train_texts, val_texts,train_labels,val_labels = train_test_split(data_texts, 
                                                                  data_labels, 
                                                                  test_size = 0.2, 
                                                                  random_state = 0 )
train_texts, test_texts,train_labels,test_labels = train_test_split(data_texts, 
                                                                  data_labels, 
                                                                  test_size = 0.01, 
                                                                  random_state = 0 )


In [15]:
train_encodings = tokenizer(train_texts,
                            truncation=True,
                            padding=True)

val_encodings = tokenizer(val_texts,
                            truncation=True,
                            padding=True)

In [19]:
len(train_encodings['input_ids'][0])

512

In [20]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] housing plans criticised by mps irreversible environmental damage will be caused by government plans to build more than one million homes in south - east england mps have warned. sustainable communities were being promoted without a real understanding of what sustainable means the environmental audit committee said. it said issues like energy needs and transport were not properly addressed. deputy prime minister john prescott said the report was completed before new initiatives were announced. he said : we are working across government especially with our colleagues at defra to create cleaner safer and greener communities while protecting and enhancing the environment. the report said there was far too little attention paid to many environmental issues including water even though supplies in parts of the south east are already too low. regulations designed to ensure energy - efficient buildings are too lax and builders routinely flout them anyway it said. financing for improving